In [1]:
import pandas as pd
import matplotlib as mpl
%matplotlib inline

In [2]:
df = pd.read_csv('vacs_train.csv', sep=';', index_col=0)

In [3]:
drops = ['id','name', 'area.name','city', 'company.id','company', 'company_link', 
         'publication_date', 'salary_currency', 'employment.name', 'schedule.name', 
         'experience.name', 'description', 'type']


def to_int(stri):
    return int(stri)


def change_emplyment(cdf):
    cdf['employment_num'] = (cdf['employment'] == 'full').apply(to_int)
    return cdf.drop('employment', axis = 1)


def change_schecule(cdf):
    type_schedule = list(cdf.groupby('schedule').count().index)
    for typ in type_schedule:
        cdf[typ + '_schedule'] = (cdf['schedule'] == typ).apply(to_int)
    cdf = cdf.drop('schedule', axis = 1)
    return cdf


def delete_dublicates(x):
    return list(dict.fromkeys(x))


def spec_to_int(string):
    spec = string.split()
    res = []
    for num in spec:
        res.append(int(float(num)))
    return delete_dublicates(res)


def add_spec(cdf):
    cdf['spec'] = cdf['specializations'].apply(spec_to_int)
    specs = []
    for arr in cdf['spec']:
        for num in arr:
            if not num in specs:
                specs.append(num)
    for i in sorted(specs):
        cdf[str(i) + '_spec'] = cdf['spec'].apply(lambda s: int(i in s))
    return cdf


def nor_experience(s):
    if s == 'between1And3':
        return 2
    if s == 'between3And6':
        return 4
    if s == 'moreThan6':
        return 8
    return 0


def change_experience(cdf):
    cdf['experience'] = cdf['experience'].apply(nor_experience)
    return cdf


def get_cur_df(df):
    df1 = df.copy()
    cdf = df1.drop(drops, axis=1)
    cdf = change_emplyment(cdf)
    cdf = change_schecule(cdf)
    cdf = add_spec(cdf)
    cdf = change_experience(cdf)
    return cdf

In [5]:
cdf = get_cur_df(df)

In [6]:
cdf

,name.lemm,salary_from,experience,key_skills,specializations,specializations.names,description.lemm,employment_num,flexible_schedule,flyInFlyOut_schedule,...,19_spec,20_spec,21_spec,22_spec,23_spec,24_spec,25_spec,26_spec,27_spec,29_spec
index,,,,,,,,,,,,,,,,,,,,,
0,менеджер туризм,40000.0,2,NaN,17.334 17.242 17.149 22.223 22.39 22.199,Продажи - Услуги для населения | Продажи - Пря...,обязанность работа турист физический юридическ...,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,помощник руководитель,40000.0,2,NaN,4.205 4.429 6.319 6.247 2.249,Административный персонал - Персональный ассис...,вакансия открывать рамка отдел строительный от...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,аналитик,30000.0,2,NaN,1.117 1.327 1.25,"Информационные технологии, интернет, телеком -...",обязанность работа рамка отдел корпоративный п...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,авиакассир,30000.0,2,NaN,22.11 21.12 17.334,"Туризм, гостиницы, рестораны - Авиабилеты | Тр...",обязанность бронирование выписка авиабилет сис...,1,0,0,...,0,0,1,1,0,0,0,0,0,0
5,бухгалтер,25000.0,4,NaN,2.43 2.335 2.463 2.337 2.469 5.42,"Бухгалтерия, управленческий учет, финансы пред...",обязанность ведение бухгалтерский налоговый уч...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150858,менеджер отдел арендный отношение,35000.0,2,NaN,17.333 17.149,Продажи - Услуги для бизнеса | Продажи - Менед...,управлять компания сбс мегамолл динамичный акт...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
150860,офис-менеджер,25000.0,4,Обеспечение жизнедеятельности офиса | Прием и ...,4.332,Административный персонал - Управляющий офисом...,обязанность обеспечение жизнедеятельность офис...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
150861,бухгалтер участок,40000.0,2,NaN,2.43,"Бухгалтерия, управленческий учет, финансы пред...",обязанность выставление счет оплата банк выпис...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
